In [1]:
from openscad1 import *

# Visualise the section logic

In [34]:
sec=pts1([[0,0,1],[20,0,1],[0,10,1],[-5,0,1],[0,-5,5],[-10.001,0,5],[0,5,1],[-5,0,1]])
sec=pts1([[2,0],[-2,0,2],[0,10,3],[-3,0]])
sec=pts1([[-20,-7.5,2.4],[5,0,2.4],[0,10,3],[15,2,70],[15,-2,3],[0,-10,2.4],
        [5,0,2.4],[1,10,1],[-1,5,7],[-20,3,90],[-20,-3,7],[-1,-5,1]])
sec=pts1([[-35,4,0],[10,8,20],[20,-5,10],[20,8,20],[10,-9,20],[10,1,0]])
sec=pts1([[-25,0],[10,5,5],[10,-3,10],[10,5,5],[10,-8,7],[10,1]])
sec=pts1([[-3,0,0],[3,0,3],[3,5,7],[-5,20,100],[8,30,20],[-11,10,5],[0,10,0]])
sec=pts1([[-5,10,3],[10,0,3],[0,20,30],[-3,20,30],[0,10,1.9],[-4,0,1.9],[0,-10,30],[-3,-20,30]])
# sec=pts1([[1,0],[-1,0,1],[0,2,1],[-1,0]])

s=10
r_l=array(sec)[:,2].tolist()
sec=array(sec)[:,:2].tolist()

p0=[sec[-1]]+sec[:-1]
p1=sec
p2=sec[1:]+[sec[0]]

p0,p1,p2=array([p0,p1,p2])
v1,v2=p0-p1,p2-p1
u1,u2=v1/norm(v1,axis=1).reshape(-1,1),v2/norm(v2,axis=1).reshape(-1,1)
p_o=cwv(p1.tolist())

theta=[(180-ang_2lineccw(p1[i],p0[i],p2[i]))/2  if p_o[i]==1 else (180-ang_2linecw(p1[i],p0[i],p2[i]))/2 for i in range(len(p0))]
th0=[theta[-1]]+theta[:-1]
th1=theta
c_p1=array([p1[i] if r_l[i]==0 else  
            array(p1[i])+(u1[i]*r_l[i]/cos(d2r(theta[i])))@[[cos(d2r((180-2*theta[i])/2)),sin(d2r((180-2*theta[i])/2))],[-sin(d2r((180-2*theta[i])/2)),cos(d2r((180-2*theta[i])/2))]]
            if p_o[i]==1 else 
            array(p1[i])+(u1[i]*r_l[i]/cos(d2r(theta[i])))@[[cos(d2r((180-2*theta[i])/2)),-sin(d2r((180-2*theta[i])/2))],[sin(d2r((180-2*theta[i])/2)),cos(d2r((180-2*theta[i])/2))]]
            for i in range(len(p0))]).tolist()

cir_1=[ 
    [p1[i].tolist()]
    if r_l[i]==0 else
    circle(r_l[i],cp=c_p1[i])
    for i in range(len(p0))]

# radiuses
r0=[r_l[-1]]+r_l[:-1]
r1=r_l
# circles
c0=[cir_1[-1]]+cir_1[:-1]
c1=cir_1
# center points
cp0=[c_p1[-1]]+c_p1[:-1]
cp1=c_p1
# orientations
p_o_0=[p_o[-1]]+p_o[:-1]
p_o_1=p_o
a=[]
for i in range(len(p0)):
    if r0[i]==0 and r1[i]==0:
        a.append([p1[i].tolist()])
    elif p_o_0[i]==-1 and r0[i]>0 and r1[i]==0:
        a.append([cir_p_t(c0[i],p1[i]),p1[i].tolist()])
    elif p_o_0[i]==1 and r0[i]>0 and r1[i]==0:
        a.append([p_cir_t(p1[i],c0[i]),p1[i].tolist()])
    elif p_o_1[i]==-1 and r0[i]==0 and r1[i]>0:
        a.append([p_cir_t(p0[i],c1[i])])
    elif p_o_1[i]==1 and r0[i]==0 and r1[i]>0:
        a.append([cir_p_t(c1[i],p0[i])])
    elif p_o_0[i]==1 and p_o_1[i]==1 and r0[i]>0 and r1[i]>0:
        if (r0[i]*tan(d2r(th0[i]))+r1[i]*tan(d2r(th1[i])))>norm(p1[i]-p0[i]):
            raise ValueError('radiuses more tha acceptable limit')
        else:
            a.append(flip(tctpf(r0[i],r1[i],cp0[i],cp1[i])[2:]))
    elif p_o_0[i]==1 and p_o_1[i]==-1 and r0[i]>0 and r1[i]>0:
        a.append(tcct(r0[i],r1[i],cp0[i],cp1[i],1))
    elif p_o_0[i]==-1 and p_o_1[i]==1 and r0[i]>0 and r1[i]>0:
        a.append(tcct(r0[i],r1[i],cp0[i],cp1[i],-1))
    elif p_o_0[i]==-1 and p_o_1[i]==-1 and r0[i]>0 and r1[i]>0:
        if (r0[i]*tan(d2r(th0[i]))+r1[i]*tan(d2r(th1[i])))>norm(p1[i]-p0[i]):
            raise ValueError('radiuses more tha acceptable limit')
        else:
            a.append(tctpf(r0[i],r1[i],cp0[i],cp1[i])[:2])

b=[0]
for i in range(len(p0)):
    if r_l[i]>0:
        b.append(b[-1]+2)
    else:
        b.append(b[-1]+1)

b=array(b[1:])-1


c=concatenate(a).tolist()
c=c if r_l[-1]==0 else c[1:]+[c[0]]
d=[]
for i in range(len(p0)):
    if r_l[i]==0:
        d.append([c[b[i]]])
    else:
        d.append( arc_2p(c[b[i]-1],c[b[i]],r_l[i],p_o[i],s))


d=concatenate(d).tolist()
d=min_d_points(d,.0001)

with open('/users/sanjeevprabhakar/openscad/trial.scad','w+') as f:
    f.write(f'''
include<dependencies2.scad>

color(["blue",.1])p_line3d({sec},.05,1);
color("magenta",.2)points({c_p1},.3);

color([.2,.2,.2,.3]) for(p={cir_1}) p_line3dc(p,.05,1);
color("green") for(p={a}) points(p,.2);
p={c};
color("cyan")for(i=[0:len(p)-1]) translate(p[i]) text(str(i),.25);
color("magenta")p_line3d({d},.15,1);

''')



In [73]:
# condition-2 r0<=line length
# c0=circle(1,[-1,1])
# p1=[0,2]
# p=cir_p_t(c0,p1)


# condition-3 r0<=line length
# c0=circle(1,[1,1])
# p1=[0,2]
# p=p_cir_t(p1,c0)

# condition-4 r1<=line length
# p1=[0,0]
# c0=circle(1,[-1,1])
# p=p_cir_t(p1,c0)

# condition-5 r1<=line length
p1=[0,0]
c0=circle(1,[1,1])
p=cir_p_t(c0,p1)




with open('/users/sanjeevprabhakar/openscad/trial.scad','w+') as f:
    f.write(f'''
include<dependencies2.scad>

color("blue")p_line3dc({c0},.02,1);
color("magenta")points({[p1]},.1);
color("green")points({[p]},.1);




''')

In [239]:
# condition-6 r0+r1<=line length
c0=circle(5,[10,10])
c1=circle(5,array([10,10])+.0001)
p=flip(tctpf(5,5,[10,10],array([10.001,10]))[2:])
p0=p[0]

# condition-7
# c0=circle(1,[1,1])
# c1=circle(1,[-1,.5])
# p=tcct(1,1,[1,1],[-1,.5],1)
# p0=p[0]

# condition-8
# c0=circle(1,[-1,1])
# c1=circle(1,[1,2])
# p=tcct(1,1,[-1,1],[1,2],-1)
# p0=p[0]


# condition-9 r0+r1<=line length
# c0=circle(1,[-1,1])
# c1=circle(1,[-1,2])
# p=tctpf(1,1,[-1,1],[-1,2])[:2]
# p0=p[0]

with open('/users/sanjeevprabhakar/openscad/trial.scad','w+') as f:
    f.write(f'''
include<dependencies2.scad>

color("blue")p_line3dc({c0},.02,1);
color("magenta")p_line3dc({c1},.02,1);

color("green")points({p},.1);
color("cyan")points({[p[0]]},.12);

''')
p

[[10.0, 15.0], [10.001, 15.0]]

In [6]:
sec=circle(10)
path=corner_radius(pts1([[2,0],[-2,0,2],[0,4,2],[-2,0]]),10)
sol=prism(sec,path)
sol1=surf_offset(sol,-1)
sol2=swp_prism_h(sol,sol1)

with open('/users/sanjeevprabhakar/openscad/trial.scad','w+') as f:
    f.write(f'''
include<dependencies2.scad>

{swp_c(sol2)}

''')